In [1]:
import gc
import glob
import json
import os
import math
from collections import namedtuple
from os.path import join as pjoin
import codecs
from multiprocessing import Pool
import multiprocessing.pool
import numpy as np
from collections import defaultdict
from itertools import groupby

import sentencepiece
from gensim import corpora, models, similarities
# from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize

In [2]:
spm = sentencepiece.SentencePieceProcessor()
spm.load(r"E:\graphsum\data\spm9998_3.model")


True

In [3]:
data = json.load(open(r"E:\graphsum\data\WikiSum_data_tfidf_paddle\train\WIKI.train.0.json", encoding="utf-8"))

In [4]:
def construct_tfidf_sim_graph_by_gensim(corpus, sim_threshold):
    """Constuct TFIDF similarity graph by Gensim package"""

    sim_graph = []
    raw_corpus = [' '.join(para) for para in corpus]

    # create English stop words list
    stoplist = set(stopwords.words('english'))
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()

    # Lowercase each document, split it by white space and filter out stopwords
    texts = [[word for word in para.lower().split() if word not in stoplist]
             for para in raw_corpus]
    # Create a set of frequent words
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1

    # stem each word
    processed_corpus = [[p_stemmer.stem(
        token) for token in text if frequency[token] > 1] for text in texts]

    dictionary = corpora.Dictionary(processed_corpus)
    bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

    # train the model
    tfidf = models.TfidfModel(bow_corpus)
    # transform the "system minors" string
    corpus_tfidf = tfidf[bow_corpus]

    for i, cor in enumerate(corpus_tfidf):
        if len(cor) == 0:
            print("The corpus_tfidf[i] is None: %s" % str(corpus_tfidf[i]))
            print(bow_corpus[i])
            # exit(1)

    index = similarities.SparseMatrixSimilarity(
        corpus_tfidf, num_features=len(dictionary))

    total = 0.
    count_large = 0.
    for i in range(len(corpus_tfidf)):
        sim = index[corpus_tfidf[i]]
        assert len(sim) == len(corpus_tfidf), "the tfidf sim is not correct!"
        sim_graph.append(sim.tolist())

        for s in sim:
            total += 1
            if s > sim_threshold:
                count_large += 1

    #print("sim_graph[0]: %s" % str(sim_graph[0]))

    return sim_graph, count_large, total

In [5]:
def transform(file):
    wikisum = json.load(open(file, encoding="utf-8"))
    datasets = []
    for ex in wikisum:

            src, tgt, src_str, tgt_str, sim_graph = ex["src"], ex["tgt"], ex["src_str"], ex["tgt_str"], ex["sim_graph"]

            src_token_ids = [list(group) + [11] for l in src for k,
                             group in groupby(l, lambda x: x == 11) if not k]

            src_filtered_short = [[spm.decode_ids(
                    [token]) for token in sent] for sent in src_token_ids]

            sim_graph, larger_s, sum_s = construct_tfidf_sim_graph_by_gensim(src_filtered_short, 0.05)

            b_data_dict = {"src": src_token_ids, "tgt": tgt,\
                           'src_str': src_str, "tgt_str": tgt_str,\
                           'sim_graph': sim_graph}   
            
            datasets.append(b_data_dict)
        
    return datasets   

In [6]:
train = transform(r"E:\graphsum\data\WikiSum_data_tfidf_paddle\train\WIKI.train.0.json")

The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(4, 1)]
The corpus_tfidf[i] is None: []
[(4, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(5, 1)]
The corpus_tfidf[i] is None: []
[(5, 1)]
The corpus_tfidf[i] is None: []
[(5, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf

...

The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(4, 1)]
The corpus_tfidf[i] is None: []
[(4, 1)]
The corpus_tfidf[i] is None: []
[(4, 1)]
The corpus_tfidf[i] is None: []
[(4, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 2)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 2)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf

In [7]:
test = transform(r"E:\graphsum\data\WikiSum_data_tfidf_paddle\test\WIKI.test.0.json")

The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(6, 1)]
The corpus_tfidf[i] is None: []
[(6, 1)]
The corpus_tfidf[i] is None: []
[(6, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 2)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 2)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(4, 1)]
The corpus_tfidf[i] is None: []
[(4, 1)]
The corpus_tfidf[i] is None: []
[(4, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(4, 1)]
The corpus_tfidf[i] is None: []
[(4, 1)]
The corpus_tfidf[i] is None: []
[(4, 1)]
The corpus_tfidf

...

The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(5, 1)]
The corpus_tfidf[i] is None: []
[(5, 1)]
The corpus_tfidf[i] is None: []
[(5, 1)]
The corpus_tfidf[i] is None: []
[(5, 1)]
The corpus_tfidf[i] is None: []
[(5, 1)]
The corpus_tfidf[i] is None: []
[(5, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(0, 3)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 2)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(5, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 2)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf

The corpus_tfidf[i] is None: []
[(3, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(1, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(0, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf[i] is None: []
[(2, 1)]
The corpus_tfidf

In [42]:
np.mean(d[1]["sim_graph"])

0.052114795768995664

In [43]:
np.mean(data[1]["sim_graph"])

0.07600643654919623

In [10]:
with open(r"E:\graphsum\data\WikiSum_data_tfidf_paddle\train\WIKI.train.sentence.0.json", "w") as f:
    json.dump(train, f)

In [11]:
with open(r"E:\graphsum\data\WikiSum_data_tfidf_paddle\test\WIKI.test.sentence.0.json", "w") as f:
    json.dump(test, f)